In [3]:
# 데이터 로드
import pandas as pd
import numpy as np

st_loc = pd.read_csv('./station_coordinate.csv', encoding='utf8')
df = pd.read_csv('./동,소재,위도,경도.csv', encoding='cp949')

In [4]:
st_loc.head(5)

,line,name,code,lat,lng
0,01호선,녹양,1908.0,37.759380,127.042292
1,01호선,남영,1002.0,37.541021,126.971300
2,01호선,용산,1003.0,37.529849,126.964561
3,01호선,노량진,1004.0,37.514219,126.942454
4,01호선,대방,1005.0,37.513342,126.926382


In [5]:
df.head(5)

,소재지,번호,위도,경도,동
0,마포구,102,37.555649,126.910629,서교동
1,마포구,103,37.554951,126.910835,망원1동
2,마포구,104,37.550629,126.914986,서교동
3,마포구,105,37.550007,126.914825,서교동
4,마포구,106,37.548645,126.912826,합정동


In [7]:
from tqdm import tqdm


df.reset_index(drop=True,inplace=True)

df_diff_rental = pd.DataFrame()
df_diff_return = pd.DataFrame()
station = []
diff_rental = []
diff_return = []
df_rank1_station_rental = []
df_rank1_diff_rental = []
df_rank1_station_return = []
df_rank1_diff_return = []

ex = pd.DataFrame()

for i in tqdm(range(len(df))):
    for j in range(len(st_loc)):
        station.append(st_loc['name'][j])
        diff_rental.append(np.sqrt((df['위도'][i]-st_loc['lat'][j])**2 +
                            (df['경도'][i]-st_loc['lng'][j])**2))
        
    ex['역'] = station
    ex['대여'] = diff_rental

    df_diff_rental['역(대여)'] = station
    df_diff_rental['차이(대여)'] = diff_rental

    df_diff_rental.sort_values('차이(대여)', inplace=True)
    df_rank1_station_rental.append(df_diff_rental.iloc[0]['역(대여)'])
    df_rank1_diff_rental.append(df_diff_rental.iloc[0]['차이(대여)'])
    df_rank2_station_rental.append(df_diff_rental.iloc[1]['역(대여)'])
    df_rank2_diff_rental.append(df_diff_rental.iloc[1]['차이(대여)'])

    station = []
    diff_rental = []
    diff_return = []
    
df['1순위(대여)'] = df_rank1_station_rental
df['1순위 차이(대여)'] = df_rank1_diff_rental

df

100%|██████████████████████████████████████████████████████████████████████████████| 2653/2653 [01:44<00:00, 25.27it/s]


,소재지,번호,위도,경도,동,1순위(대여),1순위 차이(대여)
0,마포구,102,37.555649,126.910629,서교동,망원,0.000729
1,마포구,103,37.554951,126.910835,망원1동,망원,0.001386
2,마포구,104,37.550629,126.914986,서교동,합정,0.001707
3,마포구,105,37.550007,126.914825,서교동,합정,0.001215
4,마포구,106,37.548645,126.912826,합정동,합정,0.001225
...,...,...,...,...,...,...,...
2648,영등포구,5851,37.502083,126.893188,구로5동,신도림,0.006907
2649,영등포구,5852,37.503006,126.893829,대림3동,신도림,0.006255
2650,영등포구,5853,37.522610,126.923012,여의동,여의도,0.001537
2651,영등포구,5854,37.525089,126.924080,여의동,여의도,0.003467


In [8]:
df.rename(columns = {'1순위(대여)':'대여소근처역'},inplace=True)

In [9]:
del df['소재지']
del df['위도']
del df['경도']
del df['동']
del df['1순위 차이(대여)']

In [10]:
df.drop_duplicates(inplace=True)
df

,번호,대여소근처역
0,102,망원
1,103,망원
2,104,합정
3,105,합정
4,106,합정
...,...,...
2648,5851,신도림
2649,5852,신도림
2650,5853,여의도
2651,5854,여의도


In [11]:
df.to_csv('close_station.csv', encoding='utf8')